In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Synthyra/ESMplusplus_small", trust_remote_code=True, num_labels=4)
tokenizer = model.tokenizer

In [ ]:
model

In [ ]:
sequences = ['MPRTEIN', 'MSEQWENCE']
tokenized = tokenizer(sequences, padding=True, return_tensors='pt')

labels = torch.randint(0, 4, (2,)).long() # random labels for the two sequences
tokenized['labels'] = labels
# if you are on device, make sure to move the tokenized to the device
# tokenized = {k: v.to(device) for k, v in tokenized.items()}
output = model(**tokenized)
print(output)

In [ ]:
from modeling_esm_plusplus import ESMplusplusForMaskedLM

model = ESMplusplusForMaskedLM.from_pretrained("Synthyra/ESMplusplus_small")
tokenizer = model.tokenizer
sequences = ['MPRTEIN', 'MSEQWENCE']
tokenized = tokenizer(sequences, padding=True, return_tensors='pt')

output = model(**tokenized, output_attentions=True)
att = output.attentions
len(att)

In [1]:
from modeling_fastesm import FastEsmForMaskedLM

model = FastEsmForMaskedLM.from_pretrained("Synthyra/FastESM2_650")
tokenizer = model.tokenizer
sequences = ['MPRTEIN', 'MSEQWENCE']
tokenized = tokenizer(sequences, padding=True, return_tensors='pt')

output = model(**tokenized, output_attentions=True)
att = output.attentions
len(att)

c:\Users\lhall\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


33